<a href="https://colab.research.google.com/github/ishammansoor/AI-and-Machine-Learning/blob/main/Sentimental_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

Read in data from file

In [12]:
with open("sarcasm.json", 'r') as f:
  datastore = json.load(f);

sentences = []
labels = []
urls = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

Spliting into training and testing [link text](https://)

In [13]:

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]


training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

Tokenize the data and create training and testing sequences

In [18]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [25]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 2s - loss: 0.6747 - accuracy: 0.5725 - val_loss: 0.6183 - val_accuracy: 0.7714 - 2s/epoch - 3ms/step
Epoch 2/30
625/625 - 1s - loss: 0.4564 - accuracy: 0.8243 - val_loss: 0.4033 - val_accuracy: 0.8255 - 1s/epoch - 2ms/step
Epoch 3/30
625/625 - 1s - loss: 0.3227 - accuracy: 0.8711 - val_loss: 0.3549 - val_accuracy: 0.8517 - 1s/epoch - 2ms/step
Epoch 4/30
625/625 - 1s - loss: 0.2693 - accuracy: 0.8953 - val_loss: 0.3423 - val_accuracy: 0.8566 - 1s/epoch - 2ms/step
Epoch 5/30
625/625 - 1s - loss: 0.2338 - accuracy: 0.9092 - val_loss: 0.3426 - val_accuracy: 0.8588 - 1s/epoch - 2ms/step
Epoch 6/30
625/625 - 1s - loss: 0.2066 - accuracy: 0.9210 - val_loss: 0.3522 - val_accuracy: 0.8542 - 1s/epoch - 2ms/step
Epoch 7/30
625/625 - 2s - loss: 0.1840 - accuracy: 0.9312 - val_loss: 0.3619 - val_accuracy: 0.8512 - 2s/epoch - 2ms/step
Epoch 8/30
625/625 - 1s - loss: 0.1666 - accuracy: 0.9366 - val_loss: 0.3817 - val_accuracy: 0.8448 - 1s/epoch - 2ms/step
Epoch 9/30
625/625 - 1s 

In [30]:
sentence = [
    "granny starting to fear spiders in the garden might be real",
    "the weather today is bright and sunny"
]

sequences = tokenizer.texts_to_sequences(sentence)

padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(model.predict(padded))

1/1 [==============================] - 0s 23ms/step
[[9.7767037e-01]
 [1.3948158e-04]]
